DATA IMPORT AND INITIAL FORMATTING

In [2]:
#SETTING PATHS AND IMPORTING MODULES
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import style
style.use('dark_background')
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.float_format', lambda x: '%.5f' % x)


def hdf(df, key, filename = 'data_price_qual.h5', data_columns = True, complib = 'zlib', path = path2clean, mode = 'a'):
    """
    df = DataFrame to save to hdf 
    path = Path from pathlib (from pathlib import Path)
    key = key to save the df within the df

    Saves the DF in the specified path with the keys specified

    """
    df.to_hdf(path / filename , key = key, mode = mode, data_columns = data_columns, complib = complib)
    print('The dataframe was saved to HDF under the key: ' + key + ' within the ' + filename) 


In [ ]:
#IMPORTING THE DATA 
data_blockchain     = pd.read_hdf(path2cc   / 'data_blockchain.h5',         key = 'data_blockchain') 
data_social         = pd.read_hdf(path2cc   / 'data_social_cc.h5',          key = 'data_social_cc')
data_price          = pd.read_hdf(path2cc   / 'data_price.h5',              key = 'data_price')
data_qual           = pd.read_hdf(path2bicr / 'data_qual_v2.h5',            key = 'data_qual_v2')
coin_links          = pd.read_hdf(path2cc   / 'coin_links.h5',              key = 'coins_links')
irx                 = pd.read_csv(path2irx  / '^IRX.csv')
tg                  = pd.read_hdf(path2tg   / 'data_telegram_processed.h5', key = 'data_telegram_processed')
mvda100             = pd.read_hdf(path2cc   / 'mvda100.h5',                 key = 'mvda100')

In [ ]:
#CLEANING MVDA100 INDEX DATA 
mvda100.time = pd.to_datetime(mvda100.time).dt.date
mvda100['prc_change'] = mvda100.close.pct_change()
mvda100.time = pd.to_datetime(mvda100.time).dt.date
mvda100.time = pd.to_datetime(mvda100.time)

In [ ]:
#CLEANING IRX DATA
irx.rename(columns  = {'Date':'date', 'Close' : 'rf'}, inplace = True)
irx.date            = pd.to_datetime(irx.date)
irx                 = irx[['date', 'rf']]
irx.rf              = irx.rf/100

In [ ]:
#DROPPING DUPLICATES 
data_blockchain.drop_duplicates(inplace = True)
data_social.drop_duplicates(inplace     = True)
data_price.drop_duplicates(inplace      = True)
coin_links.drop_duplicates(inplace      = True)
data_qual.drop_duplicates(inplace       = True)

In [ ]:
#RENAMING THE COLUMNS 
data_blockchain.rename(columns  = {'time':'date'}, inplace = True)
data_social.rename(columns      = {'time':'date'}, inplace = True)
data_price.rename(columns       = {'time':'date'}, inplace = True)
mvda100.rename(columns          = {'time':'date'}, inplace = True)

In [ ]:
#FOLLOWERS 
rolling_avg_dllw            = data_social.groupby('symbol').apply(lambda x: np.log(1 + x.followers.pct_change())\
                   .rolling(window = 7, min_periods = 1).mean())
log_dfollowers              = data_social.groupby('symbol').apply(lambda x: np.log(1 + x.followers.pct_change()))
data_social['growth_fllw']  = ((rolling_avg_dllw <= log_dfollowers.shift(-1)) * 1).values

rolling_avg_dllw            = data_social.groupby('symbol').apply(lambda x: np.log(1 + x.followers.pct_change())\
                .rolling(window = 7, min_periods = 1).mean())
log_dfollowers              = data_social.groupby('symbol').apply(lambda x: np.log(1 + x.followers.pct_change()))
data_social['fllw_davg']    = (log_dfollowers - rolling_avg_dllw).values
data_social['fllw_d']       = log_dfollowers.values

In [ ]:
#SETTING THE START OF THE DATA AS 2019
data_blockchain = data_blockchain.loc[data_blockchain.date  >= '2017-12-31'].reset_index(drop = True)
data_social     = data_social.loc[data_social.date          >= '2017-12-31'].reset_index(drop = True)
data_price      = data_price.loc[data_price.date            >= '2017-12-31'].reset_index(drop = True)
data_qual       = data_qual.loc[data_qual.date              >= '2018-12-31'].reset_index(drop = True)

In [ ]:
#SETTING SYMBOLS AS LOWER CASE 
data_qual.symbol    = data_qual.symbol.str.lower()
data_qual.name      = data_qual.name.str.lower() 

In [ ]:
#INSTERTING AND FORWARD FILLING THE 3 MONTH TREASURY BILLS 
data_price = pd.merge(data_price, irx, on = 'date', how = 'left')
data_price.sort_values('symbol', inplace = True)
data_price.reset_index(inplace = True, drop = True)
data_price = data_price.sort_values(['symbol', 'date']).reset_index(drop = True)
data_price.rf = data_price.groupby('symbol').apply(lambda x: x.rf.ffill()).values

In [ ]:
#SETTING INDEX AS DATE AND SYMBOL - NEEDED FOR MERGING ON INDEX 
data_blockchain.set_index(['symbol', 'date'], inplace   = True)
data_social.set_index(['symbol', 'date'],     inplace   = True)
data_price.set_index(['symbol', 'date'],      inplace   = True)
data_qual.set_index(['symbol', 'date'],       inplace   = True)

In [ ]:
#REMOVING UNNECESSARY COLUMNS
data_social.drop(columns    = ['coinid', 'fullname', 'description'], inplace = True)
data_price.drop(columns     = ['conversionType', 'conversionSymbol'], inplace = True)


In [ ]:
#SORTING INDEX OF ALL DATAFRAMES 
data_price.sort_index(level         = 0, axis = 0, inplace = True)
data_blockchain.sort_index(level    = 0, axis = 0, inplace = True)
data_price.sort_index(level         = 0, axis = 0, inplace = True)
data_social.sort_index(level        = 0, axis = 0, inplace = True)

MERGING THE DATA LIBRARIES 

In [ ]:
data_price.reset_index(inplace = True)
data_price = pd.merge(data_price, mvda100[['date', 'prc_change']], on = 'date', how = 'left').rename(columns = {'prc_change':'mktret'}).\
                      set_index(['symbol', 'date']).sort_index()

In [ ]:
#MKTCAP DATA 
data_price_mktcap = pd.merge(data_blockchain['current_supply'], data_price, right_index=True, left_index=True, how = 'right')

In [ ]:
#MERGING WITH QUAL COLUMNS 
qual_columns        = ['team_score', 'project_score']
data_qual           = data_qual.groupby(level='symbol').apply(lambda x: x.fillna(method = 'ffill', limit = 7))
data_price_qual     = pd.merge(data_price_mktcap, data_qual[qual_columns], left_index = True, right_index = True, how = 'left')

In [ ]:
#LINEAR INTERPOLATION OF SUPPLY 
supply_interpolated         = pd.DataFrame(data_price_qual.groupby(data_price_qual.index.get_level_values(1)).apply(lambda x: x['current_supply'].\
                                interpolate(method = 'linear')).dropna())
supply_interpolated.index   = supply_interpolated.index.droplevel(2)
supply_interpolated.rename(columns = {'current_supply':'supply_interpolated'}, inplace = True)

In [ ]:
#MERGING SUPPLY AND PRICE DATA 
data_price_qual = pd.merge(data_price_qual, supply_interpolated, right_index = True, left_index = True, how = 'left') 

In [ ]:
#MERGING SOCIAL, PRICE, BLOCKCHAIN DATA 
data_price_qual_soc = pd.merge(data_price_qual, data_social, right_index = True, left_index = True, how = 'left')
data_price_qual_soc = data_price_qual_soc.reset_index().set_index(['symbol', 'date'])
data_price_qual_soc = data_price_qual_soc.loc[data_price_qual_soc.index.get_level_values(1) < '2021-09-05']
data_price_qual_soc.drop(columns = ['id', 'name'], inplace = True)


In [ ]:
#CALCULATING THE MARKETCAP 
data_price_qual_soc['mktcap'] = data_price_qual_soc['current_supply'] * data_price_qual_soc['close']
data_price_qual_soc.drop(columns=['supply_interpolated', 'current_supply'], inplace = True)

In [ ]:
#FILLING IN MISSING MKTCAP VALUES WITH DATA FROM BICR 
data_qual = data_qual.reset_index().set_index(['symbol', 'date']).sort_values(['symbol', 'date'])
data_price_qual_soc = data_price_qual_soc.loc[data_price_qual_soc.index.get_level_values(1) < '2021-09-01']
data_price_qual_soc.mktcap = data_price_qual_soc['mktcap'].fillna(data_qual['market_cap'])
data_price_qual_soc.mktcap =  data_price_qual_soc.groupby(data_price_qual_soc.index.get_level_values(0)).apply(lambda x: x['mktcap'].fillna(method = 'ffill', limit = 11)).values

In [ ]:
#FORWARD FILLING THE SCORES 
data_price_qual_soc.team_score    =  data_price_qual_soc.groupby(data_price_qual_soc.index.get_level_values(0)).apply(lambda x: x['team_score'].fillna(method = 'ffill', limit = 11)).values
data_price_qual_soc.project_score =  data_price_qual_soc.groupby(data_price_qual_soc.index.get_level_values(0)).apply(lambda x: x['project_score'].fillna(method = 'ffill', limit = 11)).values

PRICE AND MARKET RELATED VARIABLES CONSTRUCTION

In [ ]:
#RESETING 0 TO NAN
data_price_qual_soc.replace(0, np.nan, inplace = True)
data_price_qual_soc = data_price_qual_soc.groupby(level='symbol').apply(lambda x: x.fillna(method = 'ffill', limit = 2))

In [ ]:
#REMOVING ALL COINS THAT HAVE A CLOSE PRICE OF 0 ON ANY DAY 
data = data_price_qual_soc.copy()
data                  = data.groupby(level = 'symbol').filter(lambda x: x['close'].notna().all())

#RETURNS
data['ret']           = pd.DataFrame(data.groupby(level = 'symbol').\
                                apply(lambda x: x['close'].pct_change()))['close'].\
                                sort_index(level = 0, axis = 0).values

#FORWARD RETURNS 
data.loc[:,'fret1d']  = data.groupby(['symbol'])['ret'].shift(-1)
data.loc[:,'fret1w']  = data.groupby(['symbol'])['ret'].\
                                apply(lambda z: z.rolling(window = 5).\
                                apply(lambda x: np.exp(np.sum(np.log(1+x)))-1)).shift(-5)
                                
data['excret'] = data.ret - data.rf

#VALUE WEIGHTS 
data.sort_values(by = ['symbol','date'], inplace = True)
data = data.groupby(level = 'symbol').filter(lambda x: x['mktcap'].notna().all())
data.loc[:,'mktcapL'] = data.groupby(['symbol'])['mktcap'].shift(1)
data.loc[:,'vw'] = data.groupby(['date'])['mktcapL'].apply(lambda x: x/sum(x))

In [ ]:
#CUTTING OFF THE RETURNS ABOVE 95% PERCENTILE 
data = data.groupby(level = 'symbol').filter(lambda x: x['ret'].mean() < data.groupby(level = 'symbol').mean().sort_values('ret', ascending = False)['ret'].quantile(0.95))

In [ ]:
data.groupby(level = 'symbol').mean().sort_values('ret', ascending = False)

In [ ]:
#GROUPING UP DATA BY SYMBOL 
data_prc_gr                  = data.groupby(level = 'symbol')

#ROLLING QUANTILES 
q1                          = data_prc_gr.apply(lambda x: x.ret.rolling(window = 90).quantile(0.01, interpolation = 'nearest'))
q5                          = data_prc_gr.apply(lambda x: x.ret.rolling(window = 90).quantile(0.05, interpolation = 'nearest'))
q10                         = data_prc_gr.apply(lambda x: x.ret.rolling(window = 90).quantile(0.10, interpolation = 'nearest'))

#ES 
es1                         = (q1  >= data_prc_gr.apply(lambda x: x.ret))
es5                         = (q5  >= data_prc_gr.apply(lambda x: x.ret))
es10                        = (q10 >= data_prc_gr.apply(lambda x: x.ret))

es5_values                  = pd.DataFrame(data.loc[es5.values].groupby(level = 'symbol').apply(lambda x: x.ret))
es5_values.rename(columns   = {'ret' : 'es5'}, inplace = True)
es5_values                  = es5_values.droplevel(1)

es10_values                 = pd.DataFrame(data.loc[es10.values].groupby(level = 'symbol').apply(lambda x: x.ret))
es10_values.rename(columns  = {'ret' : 'es10'}, inplace = True)
es10_values                 = es10_values.droplevel(1)

data                        = pd.merge(data, es5_values, right_index  = True, left_index = True, how = 'left')
data                        = pd.merge(data, es10_values, right_index = True, left_index = True, how = 'left')

data.es5                    = data.es5.groupby(level   = 'symbol').apply(lambda x: x.rolling(window = 90, min_periods = 1).mean()) * - 1
data.es10                   = data.es10.groupby(level  = 'symbol').apply(lambda x: x.rolling(window = 90, min_periods = 1).mean()) * - 1

#VaR 1%, 5%, and 10%: higher value is associated with higher downside risk , hence -1 multiplication 
data['var1']                = q1.values  * - 1 
data['var5']                = q5.values  * - 1 
data['var10']               = q10.values * - 1 


In [ ]:
#PLOTTING CORRELATION BETWEEN DOWNSIDE RISK MEASURES 
sns.clustermap(data[['es5', 'es10', 'var1', 'var5', 'var10', 'ret']].corr('spearman'), annot=True, cmap='Blues', figsize=(5, 5),);

FACTORS

In [ ]:
#TOTAL TG MESSAGES
data_tg = pd.DataFrame(tg.groupby('date').size(), columns = ['tg'])
data_tg['d_tg'] = data_tg.tg.pct_change()
data_tg.index = pd.to_datetime(data_tg.index)
data = pd.merge(data, data_tg, how = 'left', right_index = True, left_on='date')

#TOTAL CRYPTOCOMPARE VIEWS 
total_views = pd.Series(data_social.groupby(level = 'date')['total_page_views'].sum(), name = 'views')
total_views = pd.DataFrame(total_views)
total_views['d_views'] = total_views.pct_change()
total_views.reset_index(inplace = True)
total_views.index = total_views.date
total_views.drop(columns= ['date'], inplace=True)
data = pd.merge(data, total_views, left_index = True, how = 'left', right_index = True)


OTHER VARIABLES 

In [25]:
data['log_tg']      =  data.groupby(level='symbol').apply(lambda x: np.log(x.tg)).values
data['log_views']   = data.groupby(level='symbol').apply(lambda x: np.log(x.views)).values
data['vol']         = data.groupby(level = 'symbol').apply(lambda x: x.ret.rolling(window = 90).std()).values
data['log_mktcap']  = np.log(data.mktcap)
data['log_prc']     = np.log(data.close)
data = pd.merge(data.reset_index(), data.reset_index().groupby(['symbol', pd.Grouper(key='date', freq='W')]).high.max(), on = ['symbol', 'date'], 
                how = 'left')
data.rename(columns={'high_x':'high', 'high_y': 'maxdprc'}, inplace = True)


In [46]:
#CREATING LAGGED RETURNS AND INDIVIDUAL CRYPTO MOMENTUM 
ret_pivot = data.pivot(index = 'date', columns = 'symbol', values = 'close') #PIVOT TABLES WITH RETURNS 

lag_returns = pd.DataFrame()

lags = [1, 2, 5, 10, 15, 20, 40, 60]
for lag in lags:
    lag_returns[f'return_{lag}d'] = (ret_pivot
                           .pct_change(lag)
                           .stack()
                           .add(1)
                           .pow(1/lag)
                           .sub(1)
                           )
lag_returns = lag_returns.swaplevel()

mom = [2, 5, 10, 15, 20, 40, 60]
for lag in mom:
    lag_returns[f'momentum_{lag}d'] = lag_returns[f'return_{lag}d'].sub(lag_returns.return_1d)

#MERGING DATA AND LAG RETURNS, MOMENTUM
data = pd.merge(data, lag_returns, on = ['symbol', 'date'], how = 'left')


In [84]:
data.groupby('symbol').mean().sort_values('momentum_60d', ascending = True)

,high,low,open,volumefrom,close,rf,mktret,team_score,project_score,comments,...,return_20d,return_40d,return_60d,momentum_2d,momentum_5d,momentum_10d,momentum_15d,momentum_20d,momentum_40d,momentum_60d
symbol,,,,,,,,,,,,,,,,,,,,,
gxc,0.22613,0.20311,0.21096,1109387.24917,0.21194,0.01179,0.02549,29.46229,23.05547,NaN,...,0.00562,0.00507,0.00499,-0.38002,-0.41043,-0.41561,-0.41808,-0.42012,-0.41642,-0.42288
cpc,15327.08122,0.24179,0.26342,2786366.31369,0.26170,0.01179,0.02549,12.47897,9.12605,1.00000,...,-0.00276,-0.00345,-0.00362,-0.31444,-0.38064,-0.39287,-0.39623,-0.39887,-0.40615,-0.41190
credo,0.03814,0.03401,0.03599,57123.44271,0.03587,0.01179,0.02549,12.56196,2.70246,NaN,...,0.00037,-0.00005,-0.00009,-0.25973,-0.30459,-0.30806,-0.31061,-0.31282,-0.31828,-0.32319
fsn,0.76217,0.61054,0.67393,367135.16715,0.67590,0.01179,0.02549,22.36974,11.90133,2.07767,...,0.00362,0.00295,0.00275,-0.22204,-0.24990,-0.25383,-0.25398,-0.25546,-0.26012,-0.26443
sls,26.32970,23.67515,24.89790,9576.65055,24.84879,0.01179,0.02549,6.88504,11.84609,3.00000,...,-0.00031,-0.00083,-0.00089,-0.19018,-0.23102,-0.23775,-0.23952,-0.24097,-0.24544,-0.24951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
dgd,141.83221,134.21102,137.18849,14231.77529,137.57443,0.01179,0.02549,23.30297,19.88451,83.52910,...,0.00099,0.00079,0.00060,-0.00125,-0.00184,-0.00193,-0.00208,-0.00223,-0.00185,-0.00144
uno,175.38013,172.14275,173.27142,104.78897,173.58087,0.01179,0.02549,9.69323,12.44954,17.27090,...,0.00120,0.00106,0.00093,-0.00075,-0.00105,-0.00108,-0.00109,-0.00117,-0.00125,-0.00135
btc,15594.14622,14628.99834,15137.55486,60478.48764,15163.37478,0.01179,0.02549,42.26359,47.96862,288556.57985,...,0.00098,0.00104,0.00102,-0.00048,-0.00064,-0.00078,-0.00084,-0.00093,-0.00111,-0.00096


In [14]:
hdf(data, filename = 'data_clean.h5', key = 'data_clean')

In [44]:
data = pd.read_hdf(path2clean / 'data_clean.h5', key = 'data_clean')

In [ ]:
#ROLLING 252 MARKET BETA 
def rolling_regression(df, groupby, window, exog_vars, endog, monthly_basis): 
    """"
    function applies rolling regression to a df via grouping
    df = df to apply the regression to, should contain index in numbers and both the exogeneous and endogenuous variables: DATAFRAME
    groupby = list of columns by which the df should be grouped initially, first PERMNO and second DATE: LIST
    window = rolling window : INT
    exog_vars = exogenuous variables: LIST
    endog = endogenous variables: LIST
    monthly_basis = whether the merged df only has the betas of the first day of the month for each, say, stock; FALSE = DAILY PARAMS RETURNED: BOOLEAN

    returns a DF with values grouped by PERMNO or SECID or similiar and date

    """
    import statsmodels.api as sm
    from statsmodels.regression.rolling import RollingOLS
    betas = pd.DataFrame()
    initial_col = list(df.columns)
    df1 = df
    df = df.set_index(groupby).sort_index() #DONE TO MERGE THE REGRESSION OUTPUT BY THE GROUP BY VARIABLE AND THE DATE 
    df_grouped = df.groupby(groupby[0]).filter(lambda x: len(x) > window).groupby(groupby[0])

    for name,group in df_grouped: 
        exog = sm.add_constant(df_grouped.get_group(name)[exog_vars]).sort_index()
        endog = df_grouped.get_group(name)['excret'].sort_index()       
        rols = RollingOLS(endog, exog, window = window)
        rres = rols.fit()
        params = rres.params
        betas = betas.append(params)
        print(rres)
        
    betas = betas.rename(columns=lambda x: x + 'beta')
    betas_col = list(betas.columns)
    daily_data = pd.merge(df, betas, left_index=True, right_index = True)

    if monthly_basis: 
        daily_data[groupby[0]] = daily_data.index.get_level_values(0)
        daily_data[groupby[1]] = daily_data.index.get_level_values(1)
        daily_data.reset_index(inplace=True, drop = True)
        daily_data['month_year'] = daily_data[groupby[1]].dt.to_period('M')
        datax = daily_data.groupby(by = [groupby[0], 'month_year']).first()
        datay = pd.merge_ordered(df1, datax, on = initial_col)
        data_f = pd.merge(df1, datay[betas_col + groupby].groupby(groupby[0]).apply(lambda x: x.fillna(method = 'ffill')), on = groupby).reset_index()
        return data_f   
    else: 
        return daily_data
#data = data.loc[data.date <= '2021-08-31']
betas = ['mktret']
endog = ['excret']
groupby = ['symbol', 'date']
data = rolling_regression(df = data, groupby = groupby, window = 365, endog = endog, exog_vars = betas, monthly_basis = False)
